# Kafka Producer: dealer #2

In [6]:
pip install confluent_kafka --quiet

Note: you may need to restart the kernel to use updated packages.


In [7]:
from confluent_kafka import Producer, KafkaError
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
import string

topic_vehicles = "DEALER2_VEHICLES"
topic_users = "DEALER2_USERS"

a = AdminClient({'bootstrap.servers': 'kafka:9092'})
fs = a.create_topics([NewTopic(topic_vehicles, num_partitions=1, replication_factor=1),
                      NewTopic(topic_users, num_partitions=1, replication_factor=1)])

for t, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} created".format(t))
    except Exception as e:
        print("Failed to create topic {}: {}".format(t, e))

Failed to create topic DEALER2_VEHICLES: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'DEALER2_VEHICLES' already exists."}
Failed to create topic DEALER2_USERS: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'DEALER2_USERS' already exists."}


Generate and publish a new topic message every 10 to 30 seconds.

In [9]:
producerconf = {'bootstrap.servers': 'kafka:9092'}
prod = Producer(producerconf)

%4|1642175499.553|TERMINATE|rdkafka#producer-2| [thrd:app]: Producer terminating with 48 messages (6203 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


In [10]:
from csv import DictReader,reader
from datetime import datetime
import time
import random
import json

vehicles = iter(DictReader(open('vehicles.csv', 'r')))

users = {}
registered_users = []
with open('users.csv', 'r') as user_file:
    csv_reader = reader(user_file)
    header = next(csv_reader)
    for row in csv_reader:
        users[row[0]]=row[1]

In [11]:
for vehicle in iter(DictReader(open('vehicles.csv', 'r'))):
    
    if vehicle["userID"] not in registered_users:
        uid = vehicle["userID"]
        registered_users.append(uid)
        user = {"userID": uid, "name": users[uid]}
        print("USER: {}".format(user))
        prod.produce(topic=topic_users, value=json.dumps(user))
    
    vehicle["ts"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("VEHICLE: {}".format(vehicle))
    prod.produce(topic=topic_vehicles, value=json.dumps(vehicle))
    
    # Random data generation
    time.sleep(random.randint(10,30))

USER: {'userID': '3', 'name': 'Kevin Rodriguez'}
VEHICLE: {'userID': '3', 'rid': '1', 'type': 'Car', 'manufacturer': 'Audi', 'model': 'A3', 'plate': 'DFU4HJF', 'status': 'START', 'ts': '2022-01-14 15:51:44'}
USER: {'userID': '4', 'name': 'Catherine Crandell'}
VEHICLE: {'userID': '4', 'rid': '2', 'type': 'Car', 'manufacturer': 'Mercedes', 'model': 'Classe C', 'plate': '784JD93', 'status': 'START', 'ts': '2022-01-14 15:52:05'}
USER: {'userID': '1', 'name': 'Douglas Fitch'}
VEHICLE: {'userID': '1', 'rid': '3', 'type': 'Car', 'manufacturer': 'Porsche', 'model': '911', 'plate': 'UF94JF', 'status': 'START', 'ts': '2022-01-14 15:52:26'}
USER: {'userID': '2', 'name': 'William Diaz'}
VEHICLE: {'userID': '2', 'rid': '4', 'type': 'Car', 'manufacturer': 'Audi', 'model': 'A4', 'plate': 'AL3SLS', 'status': 'START', 'ts': '2022-01-14 15:52:50'}
VEHICLE: {'userID': '1', 'rid': '3', 'type': 'Car', 'manufacturer': 'Porsche', 'model': '911', 'plate': 'UF94JF', 'status': 'END', 'ts': '2022-01-14 15:53:05'